# Introduction:
Suppose you run the platelet distribution department of a blood bank. This blood bank is associated with a very critical medical center, that have a daily need for platelets. Human blood platelets are a very complicated product, because they have very short shelf life (they will be useless after 72 hours from their production) and are also very expensive to produce. The planning department of the blood bank has produced a very accurate demand estimate for the medical center, so you already know what the demand for the medical center in the next thirty days will be. Your daily production capacity is 100. The blood bank storage unit is damaged, so all produced units must head the medical center. Producing a platelet unit costs about 50 \$, so all expired units represent this amount as a loss. The blood bank has estimated that every unfulfilled demand unit cost about 500 \$.  With all this considered, estimate an optimal (or near optimal) daily shipment schedule for the next thirty days.

Hints: 
- Think about how to represent the state of the system at any point in time.
- Think about the logic of the system, how are transitions between states perfomed?
- A production schedule is represented by a 30-number list (the shipment size for each day).
- How could you measure the performance of your shipment schedule? What is the nature of the objective function?

In [ ]:
!pip install pandas numpy
import pandas as pd
import numpy as np
demand=pd.read_csv('data/demands.csv')
demand.head(30)
